In [1]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

# For importing .xml files
import xml.etree.ElementTree as ET

# For handeling dataframes
import pandas as pd
import numpy as np
import re                                  # library for regular expression operations
import string                              # for string operations

# For text preprocessing
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer as ps  # module for stemming

# For building neural netwrok models
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import transformers

# For model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import time


In [2]:
# Load the data
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [3]:
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [4]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalence = binary_labels.aggregate('sum')/1237


# Combine the x_test and y_test sets
df = text_all.merge(binary_labels, on="id", how="left")

In [5]:
df.columns

Index(['id', 'text', 'This_Disease_0', 'This_Disease_1', 'This_Disease_2',
       'This_Disease_3', 'This_Disease_4', 'This_Disease_5', 'This_Disease_6',
       'This_Disease_7', 'This_Disease_8', 'This_Disease_9', 'This_Disease_10',
       'This_Disease_11', 'This_Disease_12', 'This_Disease_13',
       'This_Disease_14', 'This_Disease_15'],
      dtype='object')

In [5]:
# Download the stopwords from NLTK
nltk.download('stopwords')

# Import the standard English stop words list from NLTK
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
additional_sw = ['admission', 'date', 'discharge', 'summary', 'diagnoses', 'diagnosis', 'history',
                    'patient', 'report', 'status', 'signed', 'attending', 'service', 'dictated', 'dictation']
all_stopwords = additional_sw + stopwords_english

In [7]:
# # Convert all words in text column to lower case in both the train and test sets
# df['text'] = [str(x).lower() for x in df['text']]

# # Remove noiseb
# df['text']  = [re.sub("(\W|\d+|\n)", " ", elem).strip() for elem in df['text']] # remove spaces, digits and line breaks

# # Remove punctuations
# def remove_punctuations(text):
#     return ' '.join(['' if (elem in string.punctuation) else elem for elem in text.split()])

# df['text'] = [remove_punctuations(elem) for elem in df['text']]


# # Remove standard English stop words
# def remove_stopwords(text):
#     return ' '.join(['' if (elem in all_stopwords) else elem for elem in text.split()])

# df['text'] = [remove_stopwords(elem) for elem in df['text']]

# # Remove words with a length of one or two
# def remove_short_words(text):
#     return ' '.join(['' if (len(elem) <= 2) else elem for elem in text.split()])

# df['text'] = [remove_short_words(elem) for elem in df['text']]


In [8]:
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in all_stopwords) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    return(text)

In [9]:
df['text'] = [preprocessing(x) for x in df['text']]

In [10]:
# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
    pred = model.predict(X)
    pred = [1 if p>=0.5 else 0 for p in pred]
    acc = np.sum(y == pred)/len(y)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred)
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [16]:
# Create the Transformer model
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_length, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = layers.Embedding(input_dim=max_length, output_dim=embedding_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.3):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embedding_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    

def transformer_model(vocab_size, embedding_dim,max_length,dropout_rate):   
    inputs = layers.Input(shape=(max_length,))
    embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embedding_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embedding_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    
    return(model)

In [17]:
# Set the number of epochs and the batch size
max_length = 557 #(mean length of texts)
epochs = 20
batch_size = 32
dropout_rate = 0.3
embedding_dim = 200
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

In [18]:
# Run the model
X = df['text']
time_1 = time.time()

for i in range(16): # Go through each disease type
    event_categorical = df['This_Disease_'+str(i)]
    dis_type = disease_types[i]
    
    time_start = time.time()   

    # Split the data 10 times into train and test sets with stratification
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25,random_state=0)
    
    j = 0
    for train_index, test_index in sss.split(X, event_categorical):   
                
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
        tokenizer = Tokenizer(num_words=5000) # Tokenize the text
        tokenizer.fit_on_texts(x_train)
        x_train = tokenizer.texts_to_sequences(x_train)
        x_test = tokenizer.texts_to_sequences(x_test)
        vocab_size = len(tokenizer.word_index) + 1 # plus the reserved index 0

        # Pad the sequences with 0's
        x_train = pad_sequences(x_train, padding='post', maxlen=max_length) 
        x_test = pad_sequences(x_test, padding='post', maxlen=max_length)

        # Fit the CNN model
        j += 1
        iteration = "iter" + str(j)
          
        time_s = time.time()  
        mymodel = transformer_model(vocab_size, embedding_dim,max_length,dropout_rate)
        mymodel.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
        time_e = time.time() - time_s

        # Collect and log evaluation metrics
        auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(mymodel, x_test, y_test)

        with open('other/CNN_Paper_no_embedding_Transformer_20epocs.csv','a') as fd:
            fd.write(f'{dis_type},{iteration},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{time_e},{vocab_size},{len(x_train)}\n')
    
    # Average metrics
    running_time = time.time() - time_start
    # Save the model performance metrics
    with open('other/CNN_Paper_no_embedding_Transformer_20epocs.csv','a') as fd:
        fd.write(f'{dis_type}, 10 iteration running time, {running_time}\n')

Epoch 1/20
29/29 [==============================] - 51s 2s/step - loss: 0.2642 - auc: 0.4853
Epoch 2/20
29/29 [==============================] - 51s 2s/step - loss: 0.2011 - auc: 0.6676
Epoch 3/20
29/29 [==============================] - 52s 2s/step - loss: 0.1687 - auc: 0.8114
Epoch 4/20
29/29 [==============================] - 53s 2s/step - loss: 0.0915 - auc: 0.9570
Epoch 5/20
29/29 [==============================] - 56s 2s/step - loss: 0.0187 - auc: 0.9995
Epoch 6/20
29/29 [==============================] - 55s 2s/step - loss: 0.0058 - auc: 1.0000
Epoch 7/20
29/29 [==============================] - 55s 2s/step - loss: 4.7257e-04 - auc: 1.0000
Epoch 8/20
29/29 [==============================] - 54s 2s/step - loss: 2.0631e-04 - auc: 1.0000
Epoch 9/20
29/29 [==============================] - 55s 2s/step - loss: 1.5849e-04 - auc: 1.0000
Epoch 10/20
29/29 [==============================] - 54s 2s/step - loss: 1.3756e-04 - auc: 1.0000
Epoch 11/20
29/29 [==============================] - 

29/29 [==============================] - 62s 2s/step - loss: 1.3430e-04 - auc: 1.0000
Epoch 12/20
29/29 [==============================] - 61s 2s/step - loss: 1.1054e-04 - auc: 1.0000
Epoch 13/20
29/29 [==============================] - 62s 2s/step - loss: 9.7931e-05 - auc: 1.0000
Epoch 14/20
29/29 [==============================] - 60s 2s/step - loss: 1.0278e-04 - auc: 1.0000
Epoch 15/20
29/29 [==============================] - 61s 2s/step - loss: 7.8826e-05 - auc: 1.0000
Epoch 16/20
29/29 [==============================] - 61s 2s/step - loss: 7.6672e-05 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 61s 2s/step - loss: 6.9855e-05 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 60s 2s/step - loss: 5.8295e-05 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 60s 2s/step - loss: 5.2314e-05 - auc: 1.0000
Epoch 20/20
29/29 [==============================] - 61s 2s/step - loss: 4.9410e-05 - auc: 1.0000
Epoch 1/20
29/29 [==============

29/29 [==============================] - 62s 2s/step - loss: 6.9885e-05 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 61s 2s/step - loss: 6.9896e-05 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 61s 2s/step - loss: 6.1424e-05 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 61s 2s/step - loss: 5.5745e-05 - auc: 1.0000
Epoch 20/20
29/29 [==============================] - 64s 2s/step - loss: 5.6796e-05 - auc: 1.0000
Epoch 1/20
29/29 [==============================] - 63s 2s/step - loss: 0.3127 - auc: 0.5192
Epoch 2/20
29/29 [==============================] - 61s 2s/step - loss: 0.2304 - auc: 0.6884
Epoch 3/20
29/29 [==============================] - 61s 2s/step - loss: 0.1725 - auc: 0.9076
Epoch 4/20
29/29 [==============================] - 63s 2s/step - loss: 0.0817 - auc: 0.9720
Epoch 5/20
29/29 [==============================] - 62s 2s/step - loss: 0.0190 - auc: 0.9989
Epoch 6/20
29/29 [==============================] - 61s 2

29/29 [==============================] - 61s 2s/step - loss: 0.0204 - auc: 0.9996
Epoch 7/20
29/29 [==============================] - 61s 2s/step - loss: 0.0106 - auc: 0.9999
Epoch 8/20
29/29 [==============================] - 61s 2s/step - loss: 7.0145e-04 - auc: 1.0000
Epoch 9/20
29/29 [==============================] - 61s 2s/step - loss: 5.3499e-04 - auc: 1.0000
Epoch 10/20
29/29 [==============================] - 61s 2s/step - loss: 2.9383e-04 - auc: 1.0000
Epoch 11/20
29/29 [==============================] - 62s 2s/step - loss: 2.1712e-04 - auc: 1.0000
Epoch 12/20
29/29 [==============================] - 61s 2s/step - loss: 1.8364e-04 - auc: 1.0000
Epoch 13/20
29/29 [==============================] - 61s 2s/step - loss: 1.6976e-04 - auc: 1.0000
Epoch 14/20
29/29 [==============================] - 61s 2s/step - loss: 1.6649e-04 - auc: 1.0000
Epoch 15/20
29/29 [==============================] - 61s 2s/step - loss: 1.4021e-04 - auc: 1.0000
Epoch 16/20
29/29 [========================

29/29 [==============================] - 61s 2s/step - loss: 1.1170e-04 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 61s 2s/step - loss: 9.9414e-05 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 61s 2s/step - loss: 8.9442e-05 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 61s 2s/step - loss: 7.8982e-05 - auc: 1.0000
Epoch 20/20
29/29 [==============================] - 60s 2s/step - loss: 7.6900e-05 - auc: 1.0000
Epoch 1/20
29/29 [==============================] - 62s 2s/step - loss: 0.4005 - auc: 0.6178
Epoch 2/20
29/29 [==============================] - 61s 2s/step - loss: 0.3222 - auc: 0.7789
Epoch 3/20
29/29 [==============================] - 61s 2s/step - loss: 0.1353 - auc: 0.9647
Epoch 4/20
29/29 [==============================] - 61s 2s/step - loss: 0.0275 - auc: 0.9988
Epoch 5/20
29/29 [==============================] - 61s 2s/step - loss: 0.0123 - auc: 0.9999
Epoch 6/20
29/29 [==============================] - 60s 2

...

29/29 [==============================] - 62s 2s/step - loss: 0.0534 - auc: 0.9986
Epoch 7/20
29/29 [==============================] - 62s 2s/step - loss: 0.0110 - auc: 1.0000
Epoch 8/20
29/29 [==============================] - 61s 2s/step - loss: 0.0047 - auc: 1.0000
Epoch 9/20
29/29 [==============================] - 62s 2s/step - loss: 0.0010 - auc: 1.0000
Epoch 10/20
29/29 [==============================] - 62s 2s/step - loss: 6.2014e-04 - auc: 1.0000
Epoch 11/20
29/29 [==============================] - 61s 2s/step - loss: 4.8285e-04 - auc: 1.0000
Epoch 12/20
29/29 [==============================] - 61s 2s/step - loss: 3.9385e-04 - auc: 1.0000
Epoch 13/20
29/29 [==============================] - 62s 2s/step - loss: 3.1819e-04 - auc: 1.0000
Epoch 14/20
29/29 [==============================] - 62s 2s/step - loss: 2.9863e-04 - auc: 1.0000
Epoch 15/20
29/29 [==============================] - 73s 3s/step - loss: 2.5861e-04 - auc: 1.0000
Epoch 16/20
29/29 [==============================] 

29/29 [==============================] - 61s 2s/step - loss: 1.1744e-04 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 61s 2s/step - loss: 1.0987e-04 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 61s 2s/step - loss: 9.4613e-05 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 60s 2s/step - loss: 9.9815e-05 - auc: 1.0000
Epoch 20/20
29/29 [==============================] - 61s 2s/step - loss: 7.5462e-05 - auc: 1.0000
Epoch 1/20
29/29 [==============================] - 63s 2s/step - loss: 0.6884 - auc: 0.6101
Epoch 2/20
29/29 [==============================] - 61s 2s/step - loss: 0.4919 - auc: 0.8466
Epoch 3/20
29/29 [==============================] - 62s 2s/step - loss: 0.2661 - auc: 0.9574
Epoch 4/20
29/29 [==============================] - 62s 2s/step - loss: 0.0974 - auc: 0.9951
Epoch 5/20
29/29 [==============================] - 61s 2s/step - loss: 0.0337 - auc: 0.9995
Epoch 6/20
29/29 [==============================] - 61s 2

29/29 [==============================] - 67s 2s/step - loss: 0.0032 - auc: 1.0000
Epoch 7/20
29/29 [==============================] - 64s 2s/step - loss: 0.0011 - auc: 1.0000
Epoch 8/20
29/29 [==============================] - 62s 2s/step - loss: 5.8612e-04 - auc: 1.0000
Epoch 9/20
29/29 [==============================] - 62s 2s/step - loss: 3.7458e-04 - auc: 1.0000
Epoch 10/20
29/29 [==============================] - 62s 2s/step - loss: 3.1797e-04 - auc: 1.0000
Epoch 11/20
29/29 [==============================] - 66s 2s/step - loss: 2.6438e-04 - auc: 1.0000
Epoch 12/20
29/29 [==============================] - 64s 2s/step - loss: 2.0339e-04 - auc: 1.0000
Epoch 13/20
29/29 [==============================] - 63s 2s/step - loss: 1.9818e-04 - auc: 1.0000
Epoch 14/20
29/29 [==============================] - 62s 2s/step - loss: 1.7479e-04 - auc: 1.0000
Epoch 15/20
29/29 [==============================] - 62s 2s/step - loss: 1.5802e-04 - auc: 1.0000
Epoch 16/20
29/29 [========================

29/29 [==============================] - 60s 2s/step - loss: 2.2219e-04 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 61s 2s/step - loss: 2.1629e-04 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 63s 2s/step - loss: 1.8308e-04 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 62s 2s/step - loss: 1.5680e-04 - auc: 1.0000
Epoch 20/20
29/29 [==============================] - 61s 2s/step - loss: 1.3051e-04 - auc: 1.0000
Epoch 1/20
29/29 [==============================] - 61s 2s/step - loss: 0.6431 - auc: 0.6306
Epoch 2/20
29/29 [==============================] - 60s 2s/step - loss: 0.5273 - auc: 0.7794
Epoch 3/20
29/29 [==============================] - 63s 2s/step - loss: 0.2898 - auc: 0.9427
Epoch 4/20
29/29 [==============================] - 62s 2s/step - loss: 0.0886 - auc: 0.9949
Epoch 5/20
29/29 [==============================] - 60s 2s/step - loss: 0.0250 - auc: 0.9997
Epoch 6/20
29/29 [==============================] - 62s 2

29/29 [==============================] - 63s 2s/step - loss: 0.0964 - auc: 0.9923
Epoch 7/20
29/29 [==============================] - 63s 2s/step - loss: 0.0184 - auc: 0.9998
Epoch 8/20
29/29 [==============================] - 63s 2s/step - loss: 0.0097 - auc: 0.9998
Epoch 9/20
29/29 [==============================] - 63s 2s/step - loss: 0.0029 - auc: 1.0000
Epoch 10/20
29/29 [==============================] - 61s 2s/step - loss: 0.0011 - auc: 1.0000
Epoch 11/20
29/29 [==============================] - 62s 2s/step - loss: 7.5402e-04 - auc: 1.0000
Epoch 12/20
29/29 [==============================] - 61s 2s/step - loss: 6.3938e-04 - auc: 1.0000
Epoch 13/20
29/29 [==============================] - 62s 2s/step - loss: 4.3730e-04 - auc: 1.0000
Epoch 14/20
29/29 [==============================] - 62s 2s/step - loss: 3.7195e-04 - auc: 1.0000
Epoch 15/20
29/29 [==============================] - 62s 2s/step - loss: 3.3827e-04 - auc: 1.0000
Epoch 16/20
29/29 [==============================] - 62

In [19]:
time_2 = time.time() - time_1  

In [20]:
time_2
with open('other/CNN_Paper_no_embedding_Transformer_20epocs.csv','a') as fd:
    fd.write(f'{dis_type}, Total running time, {time_2}\n')